# Modelling and prediction

- Input: prepared data
- Output: machine learning model, expected generalisation RMSE
- Features: This system takes the prepared dataframe and builds a machine learning model
for predicting number of cycles. Model selection, feature selection and handling missing data
are important parts of this system. You should evaluate at least 3 fundamentally different
modelling approaches before selecting the final model. We evaluate the performance of the
system by comparing the predicted number of cycles with the known number of cycles on
a validation/test data set. Specifically, the system should be evaluated with the root mean
squared error (RMSE) of predictions. The system should report the expected generalisation RMSE.

# Model

### Modules used

In [1]:
import pandas as pd
import numpy as np
import pickle

from sklearn.linear_model import LinearRegression, Ridge, ElasticNet, Lasso, SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.dummy import DummyRegressor

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

###  Read in data

In [2]:
training_set = pd.read_csv(('clean_data/training_set.csv'))
validation_set = pd.read_csv(('clean_data/validation_set.csv'))
test_set = pd.read_csv(('clean_data/test_set.csv'))
toPredict_2022 = pd.read_csv(('clean_data/toPredict_2022.csv'))

## Handle missing values

At first I thought about using the ffill() method that replace a NaN value with the value in the row above. However, a problem became apparent with that approach when I was scimming through the data manually. The problem is when its multiple rows in a row with missing data. Weather data is very time dependent and I therefore ffill() is not sufficient. A better approach would be to replace NaN values with the value for that hour the day before. 

A couple more things I will take care of is setting date and time as index again and change 'Hour', 'isWeekday', 'Month' to object type because I want to treat them like objects and not int. 

In [3]:
def handleNullValues(dataset):
    dataset = dataset.set_index('Dato + Tid') # Set date and time as index
    dataset.index = pd.to_datetime(dataset.index,format="%Y-%m-%d %H:%M:%S") # Change date and time to datetime
    dataset = dataset.groupby(dataset.index.hour).fillna(method='ffill') # Fill NaN with value same hour the day before
    dataset[['Hour', 'isWeekday', 'Month']] = dataset[['Hour', 'isWeekday', 'Month']].astype('object') # Change to object type
    return dataset

training_set = handleNullValues(training_set)
validation_set = handleNullValues(validation_set)
test_set = handleNullValues(test_set)
toPredict_2022 = handleNullValues(toPredict_2022)

## Choosing a model

First I will split each dataset into X and y

In [4]:
def splitSet(set):
    X = set.drop(['Volum'], axis=1) # Features
    y = set.Volum                   # Target
    return X,y

X_train, y_train = splitSet(training_set)
X_validation, y_validation = splitSet(validation_set)
X_test, y_test = splitSet(test_set)
X_toPredict, y_toPredict = splitSet(toPredict_2022)

### Baseline model

The problem is a regression problem and I will therefore use regression models.
I will use DummyRegressor from sklearn for the baseline model to get a general idea of what to expect.

In [5]:
dummy = DummyRegressor(strategy="mean") # Create an instance of DummyRegressor

dummy.fit(X_train, y_train) # Fit the model
y_pred = dummy.predict(X_validation) # Make a prediction
MSE = mean_squared_error(y_validation, y_pred) # Calculate MSE
RMSE = np.sqrt(MSE) # Root of MSE
    
print(dummy)
print('RMSE on test data: ', round(RMSE), '\n')

DummyRegressor()
RMSE on test data:  74 



RMSE is quite high for what im predicting. Id expect to be able to reduce it considerably. 

### Preprocess 

To improve the model I must make the data I feed it efficient and accurate. For the preprocess im going to split the features into categorical and numeric features. For the categorical features I will use OneHotEncoder. This will convert the data into binary features that is One Hot Encoded, which means that if a feature is represented by that column it receives a 1, otherwise a 0. For the numeric features I will use StandardScaler. This standardizes a feature by subtracting the mean and then scaling to unit variance.

In [6]:
def preprocess(X_train):
    categorical_features = ['Hour', 'isWeekday', 'Month'] # categorical features
    numeric_features = ['Globalstraling', 'Solskinstid', 'Lufttemperatur'] # numeric features
    
    preprocessor = ColumnTransformer([
    ('one-hot-encoder', OneHotEncoder(handle_unknown="ignore"), categorical_features), # use OneHotEncoder on categoric features
    ('standard_scaler', StandardScaler(), numeric_features)]) # use StandardScaler on numeric features
    return preprocessor

X_train_preprocessed = preprocess(X_train)

### Comparing models

Below I have chosen a handful of regression models to compare which is most suited. After the preprocess I expect them to perform alot better than the DummyRegressor. 

In [7]:
def compare_models(model):
    model_pipe = make_pipeline(X_train_preprocessed, model) # Create pipeline
    model_pipe.fit(X_train, y_train) # Fit the pipeline
    y_pred = model_pipe.predict(X_validation) # Make prediction on validation set
    MSE = mean_squared_error(y_validation, y_pred) # Calculate MSE
    RMSE = np.sqrt(MSE) # Root of MSE
    
    print(model)
    print('RMSE on validation data: ', round(RMSE), '\n')
    
# Models to test
dummy = DummyRegressor(strategy="mean")
linR = LinearRegression()
en = ElasticNet()
la = Lasso()
ri = Ridge(alpha=0.1)
sgd = SGDRegressor()
KNR = KNeighborsRegressor()
RFR = RandomForestRegressor()

regression_models = [dummy, linR, en, la, ri, sgd, KNR, RFR]

for model in regression_models:
    compare_models(model)

DummyRegressor()
RMSE on validation data:  74 

LinearRegression()
RMSE on validation data:  48 

ElasticNet()
RMSE on validation data:  64 

Lasso()
RMSE on validation data:  52 

Ridge(alpha=0.1)
RMSE on validation data:  48 

SGDRegressor()
RMSE on validation data:  48 

KNeighborsRegressor()
RMSE on validation data:  36 

RandomForestRegressor()
RMSE on validation data:  36 



As expected every model performed better. Especially two models stand out, KNeighborsRegressor and RandomForestRegressor, which performed twice as good.

### Feature selection

Moving forward I will only look at KNeighborsRegressor and RandomForestRegressor as they performed the best. To look at feature importance I will remove one feature at a time to see how the RMSE changes. Its hard to tell exactly how much each feature impacts the model. I think the numeric features, aka weatherdata, don't impact the model as much because the correlation to Volum was low. I believe the categorical features had more impact because I could see clear differences. For example difference in mean Volum for hours was significant. 

Theres for sure a better way to test this without re-writing the previous methods but its the result thats important here.

In [8]:
# Preprocess is the same as previous method except I remove one feature
def check_feature_process(X_train, feature_to_drop):
    numeric_features = ['Lufttemperatur','Globalstraling','Solskinstid']
    try:
        numeric_features.remove(feature_to_drop)
    except:
        pass
    categorical_features = ['Hour', 'isWeekday', 'Month']
    try:
        categorical_features.remove(feature_to_drop)
    except:
        pass
    
    preprocessor = ColumnTransformer([
    ('one-hot-encoder', OneHotEncoder(handle_unknown="ignore"), categorical_features),
    ('standard_scaler', StandardScaler(), numeric_features)])
    return preprocessor


def check_feature_compare(model):
    check_feature_pipe = make_pipeline(check_feature_preprocess, model) # Create pipeline
    check_feature_pipe.fit(X_train_check_feature, y_train) # Fit the pipeline
    y_pred = check_feature_pipe.predict(X_validation_check_feature) # Make prediction on validation set
    MSE = mean_squared_error(y_validation, y_pred) # Calculate MSE
    RMSE = np.sqrt(MSE) # Root of MSE
    
    print(model)
    print('RMSE on validation data: ', round(RMSE), '\n')
    

KNR = KNeighborsRegressor()
RFR = RandomForestRegressor()

best_models = [KNR, RFR]

features_to_test = ['Lufttemperatur','Globalstraling','Solskinstid', 'Hour', 'isWeekday', 'Month']

for n_feature in features_to_test:
    X_train_check_feature = X_train # Reset dataset for each feature to only remove one at a time
    X_validation_check_feature = X_validation # Reset dataset for each feature to only remove one at a time
    
    X_train_check_feature = X_train.drop([n_feature], axis=1) # Drop specified feature
    X_validation_check_feature = X_validation.drop([n_feature], axis=1) # Drop specified feature
    
    print('RMSE without feature "', n_feature,'"\n')
    check_feature_preprocess = check_feature_process(X_train_check_feature, n_feature)
    for model in best_models:
        check_feature_compare(model)


RMSE without feature " Lufttemperatur "

KNeighborsRegressor()
RMSE on validation data:  39 

RandomForestRegressor()
RMSE on validation data:  40 

RMSE without feature " Globalstraling "

KNeighborsRegressor()
RMSE on validation data:  37 

RandomForestRegressor()
RMSE on validation data:  37 

RMSE without feature " Solskinstid "

KNeighborsRegressor()
RMSE on validation data:  36 

RandomForestRegressor()
RMSE on validation data:  37 

RMSE without feature " Hour "

KNeighborsRegressor()
RMSE on validation data:  57 

RandomForestRegressor()
RMSE on validation data:  59 

RMSE without feature " isWeekday "

KNeighborsRegressor()
RMSE on validation data:  51 

RandomForestRegressor()
RMSE on validation data:  51 

RMSE without feature " Month "

KNeighborsRegressor()
RMSE on validation data:  41 

RandomForestRegressor()
RMSE on validation data:  41 



As expected, the numeric features had low impact and the categorical features was most important. The feature 'Hour' being the top factor increasing RMSE by about 20 while removed. What did surprise me what just how little the weatherdata affected the model. Removing 'Solskinstid' didn't change anything and removing 'Globalstraling' only increased rmse by 1. Only numeric feature which makes a noticeable difference when removed is Lufttemperatur which increased RMSE by 3-4.

Currently the models KNR and RFR are slow at making predictions. I will remove the features 'Solskinstid' and 'Globalstraling' and recalculate RMSE. This will make the model simpler and speed it up without losing too much accuracy. 

In [9]:
X_train_selected_features = X_train.drop(['Globalstraling','Solskinstid'], axis=1)
X_validation_selected_features = X_validation.drop(['Globalstraling','Solskinstid'], axis=1)

def preprocess_selected_features(X_train):
    numeric_features = ['Lufttemperatur']
    categorical_features = ['Hour', 'isWeekday', 'Month']

    preprocessor = ColumnTransformer([
    ('one-hot-encoder', OneHotEncoder(handle_unknown="ignore"), categorical_features),
    ('standard_scaler', StandardScaler(), numeric_features)])
    return preprocessor

X_train_selected_preprocessed = preprocess_selected_features(X_train_selected_features)

def compare_models(model):
    model_pipe = make_pipeline(X_train_selected_preprocessed, model) # Create pipeline
    model_pipe.fit(X_train_selected_features, y_train) # Fit the pipeline
    y_pred = model_pipe.predict(X_validation_selected_features) # Make prediction on validation set
    MSE = mean_squared_error(y_validation, y_pred) # Calculate MSE
    RMSE = np.sqrt(MSE) # Root of MSE
    
    print(model)
    print('RMSE on validation data: ', round(RMSE), '\n')
    
    
KNR = KNeighborsRegressor()
RFR = RandomForestRegressor()

regression_models = [KNR, RFR]

for model in regression_models:
    compare_models(model)

KNeighborsRegressor()
RMSE on validation data:  37 

RandomForestRegressor()
RMSE on validation data:  38 



Removing 'Globalstraling' and 'Solskinstid' only increased RMSE with KNR by 1. I will choose KNR as the best model because it now has the lowest RMSE and it makes predictions faster as well. Now that I have selected my model I will calculate RMSE on the test set and save the mode. I expect RMSE to be the same on the test set as the validation set. 

In [13]:
best_model = make_pipeline(X_train_selected_preprocessed, KNR) # Create pipeline
best_model.fit(X_train_selected_features, y_train) # Fit the pipeline

X_test_selected_features = X_test.drop(['Globalstraling','Solskinstid'], axis=1) # Drop bad features

y_pred = best_model.predict(X_test_selected_features) # Make prediction on validation set
MSE = mean_squared_error(y_test, y_pred) # Calculate MSE on test data
RMSE = np.sqrt(MSE) # Root of MSE
    
print(KNR)
print('RMSE on test data: ', round(RMSE), '\n')

pickle.dump(best_model, open('model.pkl', 'wb')) # Save the best model

KNeighborsRegressor()
RMSE on test data:  27 



## Predictions

I will then use this model to predict Volum for 2022.

In [11]:
volum_prediction = best_model.predict(X_toPredict) # Make prediction
toPredict_2022.Volum = volum_prediction # Fill Volum column with predictions
toPredict_2022.Volum = toPredict_2022.Volum.round() # Round to whole numbers
toPredict_2022.to_csv('predictions.csv') # Save as csv file